MICE -> MULTIVARIATE IMPUTATION by CHAINED EQUATIONS 

Assumptions:
1. Sure when data is Missing At Random (can predict the value of missing column by using other columns)

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

C:\Users\rohit\AppData\Local\Temp\ipykernel_9452\1083700189.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [123]:
df = np.round(pd.read_csv('50_Startups.csv')[['R&D Spend','Administration','Marketing Spend','Profit']]/10000)
np.random.seed(9)
df = df.sample(5)
df

,R&D Spend,Administration,Marketing Spend,Profit
21,8.0,15.0,30.0,11.0
37,4.0,5.0,20.0,9.0
2,15.0,10.0,41.0,19.0
14,12.0,16.0,26.0,13.0
44,2.0,15.0,3.0,7.0


In [124]:
df = df.iloc[:,0:-1]
df

,R&D Spend,Administration,Marketing Spend
21,8.0,15.0,30.0
37,4.0,5.0,20.0
2,15.0,10.0,41.0
14,12.0,16.0,26.0
44,2.0,15.0,3.0


In [125]:
df.iloc[1,0] = np.NaN
df.iloc[3,1] = np.NaN
df.iloc[-1,-1] = np.NaN
df.head()

C:\Users\rohit\AppData\Local\Temp\ipykernel_9848\4209224845.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[1,0] = np.NaN
C:\Users\rohit\AppData\Local\Temp\ipykernel_9848\4209224845.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[3,1] = np.NaN
C:\Users\rohit\AppData\Local\Temp\ipykernel_9848\4209224845.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[-1,-1] = np.NaN


,R&D Spend,Administration,Marketing Spend
21,8.0,15.0,30.0
37,NaN,5.0,20.0
2,15.0,10.0,41.0
14,12.0,NaN,26.0
44,2.0,15.0,NaN


Iterative Process Begins

In [126]:
df0 = pd.DataFrame()

df0['R&D Spend'] = df['R&D Spend'].fillna(df['R&D Spend'].mean())
df0['Administration'] = df['Administration'].fillna(df['Administration'].mean())
df0['Marketing Spend'] = df['Marketing Spend'].fillna(df['Marketing Spend'].mean())

In [127]:
df1 = df0.copy()
df1.iloc[1,0] = np.NaN
df1

,R&D Spend,Administration,Marketing Spend
21,8.0,15.00,30.00
37,NaN,5.00,20.00
2,15.0,10.00,41.00
14,12.0,11.25,26.00
44,2.0,15.00,29.25


In [128]:
X = df1.iloc[[0,2,3,4],1:3]
X

,Administration,Marketing Spend
21,15.00,30.00
2,10.00,41.00
14,11.25,26.00
44,15.00,29.25


In [129]:
y = df1.iloc[[0,2,3,4],0]
y

21     8.0
2     15.0
14    12.0
44     2.0
Name: R&D Spend, dtype: float64

In [130]:
lr = LinearRegression()
lr.fit(X,y)
lr.predict(df1.iloc[1,1:].values.reshape(1,2))

C:\Users\rohit\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([23.14158651])

In [131]:
df1.iloc[1,0] = 23.14
df1

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,23.14,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.25,26.00
44,2.00,15.00,29.25


In [132]:
df1.iloc[3,1] = np.NaN
df1

,R&D Spend,Administration,Marketing Spend
21,8.00,15.0,30.00
37,23.14,5.0,20.00
2,15.00,10.0,41.00
14,12.00,NaN,26.00
44,2.00,15.0,29.25


In [133]:
# Use last 3 rows to build a model and use the first for prediction
X = df1.iloc[[0,1,2,4],[0,2]]
X

,R&D Spend,Marketing Spend
21,8.00,30.00
37,23.14,20.00
2,15.00,41.00
44,2.00,29.25


In [134]:
y = df1.iloc[[0,1,2,4],1]
y

21    15.0
37     5.0
2     10.0
44    15.0
Name: Administration, dtype: float64

In [135]:
lr = LinearRegression()
lr.fit(X,y)
lr.predict(df1.iloc[3,[0,2]].values.reshape(1,2))

C:\Users\rohit\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([11.06331285])

In [136]:
df1.iloc[3,1] = 11.06
df1

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,23.14,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.06,26.00
44,2.00,15.00,29.25


In [137]:
# Remove the col3 imputed value
df1.iloc[4,-1] = np.NaN
df1

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.0
37,23.14,5.00,20.0
2,15.00,10.00,41.0
14,12.00,11.06,26.0
44,2.00,15.00,NaN


In [138]:
# Use last 3 rows to build a model and use the first for prediction
X = df1.iloc[0:4,0:2]
X

,R&D Spend,Administration
21,8.00,15.00
37,23.14,5.00
2,15.00,10.00
14,12.00,11.06


In [139]:
y = df1.iloc[0:4,-1]
y

21    30.0
37    20.0
2     41.0
14    26.0
Name: Marketing Spend, dtype: float64

In [140]:
lr = LinearRegression()
lr.fit(X,y)
lr.predict(df1.iloc[4,0:2].values.reshape(1,2))

C:\Users\rohit\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([31.56351448])

In [141]:
df1.iloc[4,-1] = 31.56
df1

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,23.14,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.06,26.00
44,2.00,15.00,31.56


In [142]:
df1 # 1st iteration complete

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,23.14,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.06,26.00
44,2.00,15.00,31.56


In [143]:
# Subtract 0th iteration from 1st iteration
df1 - df0

,R&D Spend,Administration,Marketing Spend
21,0.00,0.00,0.00
37,13.89,0.00,0.00
2,0.00,0.00,0.00
14,0.00,-0.19,0.00
44,0.00,0.00,2.31


In [144]:
df2 = df1.copy()
df2.iloc[1,0] = np.NaN
df2

,R&D Spend,Administration,Marketing Spend
21,8.0,15.00,30.00
37,NaN,5.00,20.00
2,15.0,10.00,41.00
14,12.0,11.06,26.00
44,2.0,15.00,31.56


In [145]:
X = df2.iloc[[0,2,3,4],1:3]
y = df2.iloc[[0,2,3,4],0]

lr = LinearRegression()
lr.fit(X,y)
lr.predict(df2.iloc[1,1:].values.reshape(1,2))

C:\Users\rohit\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([23.78627207])

In [146]:
df2.iloc[1,0] = 23.78
df2.iloc[3,1] = np.NaN
X = df2.iloc[[0,1,2,4],[0,2]]
y = df2.iloc[[0,1,2,4],1]

lr = LinearRegression()
lr.fit(X,y)
lr.predict(df2.iloc[3,[0,2]].values.reshape(1,2))

C:\Users\rohit\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([11.22020174])

In [147]:
df2.iloc[3,1] = 11.22
df2.iloc[4,-1] = np.NaN

X = df2.iloc[0:4,0:2]
y = df2.iloc[0:4,-1]

lr = LinearRegression()
lr.fit(X,y)
lr.predict(df2.iloc[4,0:2].values.reshape(1,2))

C:\Users\rohit\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([38.87979054])

In [148]:
df2.iloc[4,-1] = 31.56
df2 # 2nd iteration ends

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,23.78,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.22,26.00
44,2.00,15.00,31.56


In [149]:
df2 - df1

,R&D Spend,Administration,Marketing Spend
21,0.00,0.00,0.0
37,0.64,0.00,0.0
2,0.00,0.00,0.0
14,0.00,0.16,0.0
44,0.00,0.00,0.0


In [150]:
df3 = df2.copy()
df3.iloc[1,0] = np.NaN
df3

,R&D Spend,Administration,Marketing Spend
21,8.0,15.00,30.00
37,NaN,5.00,20.00
2,15.0,10.00,41.00
14,12.0,11.22,26.00
44,2.0,15.00,31.56


In [151]:
X = df3.iloc[[0,2,3,4],1:3]
y = df3.iloc[[0,2,3,4],0]

lr = LinearRegression()
lr.fit(X,y)
lr.predict(df3.iloc[1,1:].values.reshape(1,2))

C:\Users\rohit\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([24.57698058])

In [152]:
df3.iloc[1,0] = 24.57
df3.iloc[3,1] = np.NaN
X = df3.iloc[[0,1,2,4],[0,2]]
y = df3.iloc[[0,1,2,4],1]

lr = LinearRegression()
lr.fit(X,y)
lr.predict(df3.iloc[3,[0,2]].values.reshape(1,2))

C:\Users\rohit\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([11.37282844])

In [156]:
df3.iloc[3,1] = 11.37
df3.iloc[4,-1] = np.NaN

X = df3.iloc[0:4,0:2]
y = df3.iloc[0:4,-1]

lr = LinearRegression()
lr.fit(X,y)
lr.predict(df3.iloc[4,0:2].values.reshape(1,2))

C:\Users\rohit\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([45.53976417])

In [157]:
df3.iloc[4,-1] = 45.53
df2.iloc[3,1] = 11.22
df3

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,24.57,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.37,26.00
44,2.00,15.00,45.53


In [158]:
df3 - df2

,R&D Spend,Administration,Marketing Spend
21,0.00,0.00,0.00
37,0.79,0.00,0.00
2,0.00,0.00,0.00
14,0.00,0.15,0.00
44,0.00,0.00,13.97


In [2]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [15]:
df = pd.DataFrame({
    'age': [25,27,29,31,33,np.nan,37,39,41,np.nan,45],
    'experience': [np.nan, 3,5,7,9,11,13,16,np.nan,19,21],
    'salary': [50, np.nan, 110,140,170,200,230,260,np.nan,320,350],
    'purchased' : [0,1,1,0,1,0,0,1,1,0,0]
})
df

,age,experience,salary,purchased
0,25.0,NaN,50.0,0
1,27.0,3.0,NaN,1
2,29.0,5.0,110.0,1
3,31.0,7.0,140.0,0
4,33.0,9.0,170.0,1
5,NaN,11.0,200.0,0
6,37.0,13.0,230.0,0
7,39.0,16.0,260.0,1
8,41.0,NaN,NaN,1
9,NaN,19.0,320.0,0


In [16]:
X = df.drop('purchased', axis=1)
y = df['purchased']

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [18]:
lr = LinearRegression()
imp = IterativeImputer(estimator=lr, tol=1e-10, max_iter=1, verbose=2, imputation_order='roman')
imp.fit_transform(X_train)

[IterativeImputer] Completing matrix with shape (8, 3)
[IterativeImputer] Ending imputation round 1/1, elapsed time 0.01
[IterativeImputer] Change: 83.718083137517, scaled tolerance: 2.6e-08 


C:\Users\rohit\AppData\Roaming\Python\Python311\site-packages\sklearn\impute\_iterative.py:801: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


array([[ 25.        ,   0.53793922,  50.        ],
       [ 27.        ,   3.        ,  81.99620258],
       [ 29.        ,   5.        , 110.        ],
       [ 31.        ,   7.        , 140.        ],
       [ 33.        ,   9.        , 170.        ],
       [ 34.42116416,  11.        , 200.        ],
       [ 37.        ,  13.        , 230.        ],
       [ 39.        ,  16.        , 260.        ]])

In [19]:
X_train.mean()

age            31.571429
experience      9.142857
salary        165.714286
dtype: float64

In [20]:
# For the inital imputation of test set, the missing values in test set will be filled 
# with the means of the respective columns in the train set
X_test

,age,experience,salary
8,41.0,NaN,NaN
9,NaN,19.0,320.0
10,45.0,21.0,350.0


In [21]:
imp.imputation_sequence_

[_ImputerTriplet(feat_idx=0, neighbor_feat_idx=array([1, 2]), estimator=LinearRegression()),
 _ImputerTriplet(feat_idx=1, neighbor_feat_idx=array([0, 2]), estimator=LinearRegression()),
 _ImputerTriplet(feat_idx=2, neighbor_feat_idx=array([0, 1]), estimator=LinearRegression())]

In [22]:
print(imp.imputation_sequence_[0][2].coef_)
print(imp.imputation_sequence_[0][2].intercept_)

[0.46068289 0.04777397]
19.798858287238648


In [23]:
19*0.46068289 + 320*0.04777397 + 19.798858287238644

43.83950359723865

In [24]:
imp.transform(X_test)

[IterativeImputer] Completing matrix with shape (3, 3)
[IterativeImputer] Ending imputation round 1/1, elapsed time 0.00


array([[ 41.        ,  17.34984572, 289.89202243],
       [ 43.83950373,  19.        , 320.        ],
       [ 45.        ,  21.        , 350.        ]])

In [25]:
print(imp.imputation_sequence_[1][2].coef_)
print(imp.imputation_sequence_[1][2].intercept_)

[1.02317467 0.00381208]
-25.23203142007218


In [26]:
1.02317467*41 + 0.00381208 *165.71 - 25.23203142007218

17.349829826727817

In [27]:
print(imp.imputation_sequence_[2][2].coef_)
print(imp.imputation_sequence_[2][2].intercept_)

[9.91419532 4.81517967]
-200.1326100334474


In [28]:
9.91419532 *41 + 4.81517967*17.349 - 200.1326100334477

289.8879501813823

In [3]:
df = pd.DataFrame({
    'age': [25,27,29,31,33,np.nan],
    'experience': [np.nan, 3,5,7,9,11],
    'salary': [50, np.nan, 110,140,170,200],
    'purchased' : [0,1,1,0,1,0]
})
df

,age,experience,salary,purchased
0,25.0,NaN,50.0,0
1,27.0,3.0,NaN,1
2,29.0,5.0,110.0,1
3,31.0,7.0,140.0,0
4,33.0,9.0,170.0,1
5,NaN,11.0,200.0,0


In [5]:
X = df.drop('purchased', axis=1)
y = df['purchased']

In [6]:
X.corr()

,age,experience,salary
age,1.0,1.0,1.0
experience,1.0,1.0,1.0
salary,1.0,1.0,1.0


In [8]:
lr = LinearRegression()
imp = IterativeImputer(estimator=lr, verbose=2, max_iter=30, tol=1e-10, imputation_order='roman')

In [9]:
imp.fit_transform(X)

[IterativeImputer] Completing matrix with shape (6, 3)
[IterativeImputer] Ending imputation round 1/30, elapsed time 0.02
[IterativeImputer] Change: 61.225189877145056, scaled tolerance: 2e-08 
[IterativeImputer] Ending imputation round 2/30, elapsed time 0.02
[IterativeImputer] Change: 7.9637678910956, scaled tolerance: 2e-08 
[IterativeImputer] Ending imputation round 3/30, elapsed time 0.03
[IterativeImputer] Change: 0.7509179143104063, scaled tolerance: 2e-08 
[IterativeImputer] Ending imputation round 4/30, elapsed time 0.04
[IterativeImputer] Change: 0.01311646776996156, scaled tolerance: 2e-08 
[IterativeImputer] Ending imputation round 5/30, elapsed time 0.05
[IterativeImputer] Change: 0.000814252648837055, scaled tolerance: 2e-08 
[IterativeImputer] Ending imputation round 6/30, elapsed time 0.06
[IterativeImputer] Change: 3.9720598692838394e-05, scaled tolerance: 2e-08 
[IterativeImputer] Ending imputation round 7/30, elapsed time 0.07
[IterativeImputer] Change: 2.14128991160

array([[ 25.,   1.,  50.],
       [ 27.,   3.,  80.],
       [ 29.,   5., 110.],
       [ 31.,   7., 140.],
       [ 33.,   9., 170.],
       [ 35.,  11., 200.]])

In [10]:
corr_values = [0.9, 0.5, 0.8, 0.4, 0.1]
corr_values

[0.9, 0.5, 0.8, 0.4, 0.1]

In [11]:
np.sum(corr_values)

2.7

In [12]:
0.9/2.7

0.3333333333333333

In [13]:
from sklearn.preprocessing import normalize
probs = normalize([corr_values], norm='l1')
probs = probs.ravel()
probs,probs.sum()

(array([0.33333333, 0.18518519, 0.2962963 , 0.14814815, 0.03703704]), 1.0)

In [16]:
np.random.choice([1,2,3,4,5], 2, replace=False, p=probs)

array([2, 3])